# Upload PDFs to a Vector Database

## Overview
This notebook will guide you through uploading a sample PDF dataset to a vector database. 
You should already have a sample [Milvus](https://milvus.io/docs/install_standalone-docker.md) vector database setup from the Workbench project, which is setup to run at port `19530`. 

In [ ]:
!pip install pypdf

## Unzip Dataset
The dataset used in this example is pdf files containing NVIDIA blogs and press releases. These PDF files have been scraped and stored in `../data/corp-comms-dataset.zip`.

In [ ]:
!unzip -n ../data/corp-comms-dataset.zip -d ../data/

## Setup NVIDIA Embedding Model
This model, [embed-qa-4](https://build.nvidia.com/nvidia/embed-qa-4), is a fine-tuned E5-large model deployed as a NIM and hosted on the [NVIDIA API catalog](https://build.nvidia.com/). 


*⚠️* Be sure to populate config variables for the app!

In [ ]:
from chain_server.configuration import config
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

# "nvapi-xxx" is the NVIDIA API KEY format. If you have not configured this variable, be sure to do so. 
embedding_model = NVIDIAEmbeddings(
    model=config.embedding_model.name, 
    api_key=config.nvidia_api_key, 
    truncate="END"
)

## Setup Milvus Vector Database
[Milvus](https://milvus.io/docs/install_standalone-docker.md) should already be running through NVIDIA Workbench.  Milvus is a database that stores, indexes, and manages massive embedding vectors.

In [ ]:
print(config.milvus.collection_name)

In [ ]:
from langchain_community.vectorstores.milvus import Milvus

vector_store = Milvus(
    embedding_function=embedding_model,
    connection_args={"uri": config.milvus.url},
    collection_name=config.milvus.collection_name,
    auto_id=True,
)

## Upload PDFs to Milvus Vector Database

In [ ]:
import glob

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def upload_document(file_path):
    loader = PyPDFLoader(str(file_path))
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter()
    all_splits = text_splitter.split_documents(data)
    vector_store.add_documents(documents=all_splits)

    return f"uploaded {file_path}"

def upload_pdf_files(folder_path, num_files):
    i = 0
    for file_path in glob.glob(f"{folder_path}/*.pdf"):
        print(upload_document(file_path))
        i += 1
        if i >= num_files:
            break

In [ ]:
NUM_DOCS_TO_UPLOAD=10
upload_pdf_files("../data/dataset", NUM_DOCS_TO_UPLOAD)

In [ ]:
query = "How is NVIDIA working with Mercedes Benz?"
docs = vector_store.similarity_search(query)
print(docs[0])